In [1]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
import pandas as pd
import numpy as np
import os
import torch
import pandas as pd
import numpy as np
from skimage import io, transform
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import cv2
from torch.utils.data.sampler import SubsetRandomSampler
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models

In [6]:
!pip3 install googledrivedownloader
from google_drive_downloader import GoogleDriveDownloader as gdd

gdd.download_file_from_google_drive(file_id='0BxYys69jI14kYVM3aVhKS1VhRUk',dest_path='./UTKFace.tar.gz')
import tarfile
tar = tarfile.open("./UTKFace.tar.gz")
tar.extractall()
tar.close()

  Using cached https://files.pythonhosted.org/packages/7e/41/d59b2a5fcc7afeb40f23091694bd6e6a63ad118c93f834353ee5100285d5/googledrivedownloader-0.3-py2.py3-none-any.whl


In [ ]:
class gender_dataset(Dataset):
  def __init__(self,label,image_dir,transform = None):
    self.data = pd.read_csv(label)
    self.img_dir = image_dir
    self.transform = transform
    self.id = self.data.iloc[:,1:2].values
  def __len__(self):
        return len(self.id)
  def __getitem__(self,idx):
    img_name = os.path.join(self.img_dir,self.id[idx,0])
    image = cv2.imread(img_name)
    gender1 = self.data.iloc[:,3:4].values
    gender = gender1[idx,0]
    if self.transform:
      image = self.transform(image)
    return image,gender
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224,224)),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize([0.39125082, 0.45353994, 0.5837345 ], [0.20129617, 0.21080811, 0.24163313])
    ])
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
gender_data = gender_dataset('./agedetect/label.csv','./UTKFace',transform)
test_size = 0.2
valid_size = 0.1
data_len = len(gender_data)
indices = list(range(data_len))
np.random.shuffle(indices)
split1 = int(np.floor(valid_size * data_len))
split2 = int(np.floor(test_size * data_len))
split2 = split2+split1
valid_idx , test_idx, train_idx = indices[:split1], indices[split1:split2] , indices[split2:]
train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)
test_sampler = SubsetRandomSampler(test_idx)
train_loader = DataLoader(gender_data, batch_size=128, sampler=train_sampler)
valid_loader = DataLoader(gender_data, batch_size=128 , sampler=valid_sampler)
test_loader = DataLoader(gender_data, batch_size=128 , sampler=test_sampler)
classes = ['Male','Female']
dataloaders = {'train':train_loader,'val':valid_loader}
dataset_sizes = {'train':16597,'test':4741,'val':2370}


resnet50 = models.resnet50(pretrained=True)
num_ftrs = resnet50.fc.in_features
resnet50.classifier = nn.Sequential(nn.Linear(num_ftrs,1),nn.Sigmoid())
resnet50 = resnet50.to(device)
criterion = nn.CrossEntropyLoss()
optimizer_ft = optim.SGD(resnet50.parameters(), lr=0.0001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)
def train_model(model, criterion, optimizer, scheduler, num_epochs=10):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                scheduler.step()
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    torch.save(model,'/classifier')
    return model
model_ft = train_model(resnet50, criterion, optimizer_ft, exp_lr_scheduler,num_epochs=10)

Epoch 0/9
----------
train Loss: 2.3946 Acc: 0.5434
val Loss: 0.5332 Acc: 0.7700

Epoch 1/9
----------
train Loss: 0.4702 Acc: 0.7951
val Loss: 0.4268 Acc: 0.8046

Epoch 2/9
----------
train Loss: 0.3993 Acc: 0.8303
val Loss: 0.3872 Acc: 0.8329

Epoch 3/9
----------
